In [2]:
# System libraries:
%load_ext autoreload
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
import string
from collections import Counter
import matplotlib.pyplot as plt
import re
from polyglot.text import Text as T
InteractiveShell.ast_node_interactivity = "all"

In [10]:
data_local = pd.read_csv('data/processed/integrum_local.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
data_federal = pd.read_csv('data/processed/integrum_federal.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
data_local.iloc[:5]
data_federal.iloc[:5]


,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year
0,1,01.11.2003,Известия (Москва),Известия,"Владимир ВУЙТЕК, главный тренер хоккейного клу...",О чешском тренере Владимире ВУЙТЕКЕ нынче прин...,"[владимир, вуйтек, главный, тренер, хоккейный,...","[о, чешский, тренер, владимир, вуйтек, нынче, ...","['О', 'чешском', 'тренере', 'Владимире', 'ВУЙТ...","['Владимир', 'ВУЙТЕК', 'главный', 'тренер', 'х...",2003
1,2,03.11.2003,Известия (Москва),Известия,Судьба Маргариты,В минувшее воскресенье исполнилось 110 лет со ...,"[судьба, маргарита]","[в, минувший, воскресение, исполняться, год, д...","['В', 'минувшее', 'воскресенье', 'исполнилось'...","['Судьба', 'Маргариты']",2003
2,3,04.11.2003,Известия (Москва),Известия,Девушка в кокошнике - лицо России,В минувшие выходные в Греции завершилась 19-я ...,"[девушка, кокошник, лицо, россия]","[в, минувший, выходной, греция, завершаться, 1...","['В', 'минувшие', 'выходные', 'Греции', 'завер...","['Девушка', 'кокошнике', 'лицо', 'России']",2003
3,4,05.11.2003,Известия (Москва),Известия,Не прошло и полвека,Сотрудники МИД Финляндии разыскали живого свид...,"[не, проходить, полвека]","[сотрудник, мид, финляндия, разыскивать, живой...","['Сотрудники', 'МИД', 'Финляндии', 'разыскали'...","['Не', 'прошло', 'полвека']",2003
4,5,06.11.2003,Известия (Москва),Известия,ХРОНИКА,EXXON MOBIL НАМЕРЕНА РАЗВИВАТЬ СОТРУДНИЧЕСТВО ...,[хроника],"[EXXON, MOBIL, намерен, развивать, сотрудничес...","['EXXON', 'MOBIL', 'НАМЕРЕНА', 'РАЗВИВАТЬ', 'С...",['ХРОНИКА'],2003


,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year
0,25380,05.06.2000,Деловой Петербург,Деловой Петербург,Таможню разблокировали,Начальник Выборгской таможни Иван Иоч дал устн...,"[таможня, разблокировать]","[начальник, выборгский, таможня, иван, иоч, да...","['Начальник', 'Выборгской', 'таможни', 'Иван',...","['Таможню', 'разблокировали']",2000
1,25381,05.06.2000,Деловой Петербург,Деловой Петербург,Чем хуже-тем лучше,Только замедление темпов роста американской эк...,"[что, плохо]","[только, замедление, темп, рост, американский,...","['Только', 'замедление', 'темпов', 'роста', 'а...","['Чем', 'хуже-тем']",2000
2,25382,09.06.2000,Деловой Петербург,Деловой Петербург,&lt;Газпром&gt; протянул ветку,ОАО &lt;Газпром&gt; завершило длившееся более ...,"[lt, газпром, gt, протягивать, ветка]","[оао, lt, газпром, gt, завершать, длиться, год...","['ОАО', 'lt', 'Газпром', 'gt', 'завершило', 'д...","['lt', 'Газпром', 'gt', 'протянул', 'ветку']",2000
3,25383,09.06.2000,Деловой Петербург,Деловой Петербург,Бизнесмены дрались клюшками,Во Дворце спорта СКА в хоккей играли на-стоящи...,"[бизнесмен, драться, клюшка]","[во, дворец, спорт, ска, хоккей, играть, на, м...","['Во', 'Дворце', 'спорта', 'СКА', 'хоккей', 'и...","['Бизнесмены', 'дрались', 'клюшками']",2000
4,25384,14.06.2000,Деловой Петербург,Деловой Петербург,Kvaerner исчез с карты области,На собрании акционеров ОАО &lt;Квернер Выборг ...,"[Kvaerner, исчезать, карта, область]","[на, собрание, акционер, оао, lt, квернер, выб...","['На', 'собрании', 'акционеров', 'ОАО', 'lt', ...","['Kvaerner', 'исчез', 'карты', 'области']",2000


In [4]:

text = T("это очень плохо. А это намного лучше, даже хорошо!",hint_language_code='ru')
text.polarity

0.3333333333333333

In [11]:
def get_sentiment(content):
    try:
        text = T(content, hint_language_code='ru')
        return text.polarity
    except:
        return 0.0

In [12]:
data_local['sentiment'] = data_local.content.apply(get_sentiment)
data_local.to_csv('data/processed/integrum_local_sentiment.csv')
data_federal['sentiment'] = data_federal.content.apply(get_sentiment)
data_federal.to_csv('data/processed/integrum_federal_sentiment.csv')

In [15]:
data_federal

,id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year,sentiment
0,1,01.11.2003,Известия (Москва),Известия,"Владимир ВУЙТЕК, главный тренер хоккейного клу...",О чешском тренере Владимире ВУЙТЕКЕ нынче прин...,"[владимир, вуйтек, главный, тренер, хоккейный,...","[о, чешский, тренер, владимир, вуйтек, нынче, ...","['О', 'чешском', 'тренере', 'Владимире', 'ВУЙТ...","['Владимир', 'ВУЙТЕК', 'главный', 'тренер', 'х...",2003,0.257143
1,2,03.11.2003,Известия (Москва),Известия,Судьба Маргариты,В минувшее воскресенье исполнилось 110 лет со ...,"[судьба, маргарита]","[в, минувший, воскресение, исполняться, год, д...","['В', 'минувшее', 'воскресенье', 'исполнилось'...","['Судьба', 'Маргариты']",2003,0.298969
2,3,04.11.2003,Известия (Москва),Известия,Девушка в кокошнике - лицо России,В минувшие выходные в Греции завершилась 19-я ...,"[девушка, кокошник, лицо, россия]","[в, минувший, выходной, греция, завершаться, 1...","['В', 'минувшие', 'выходные', 'Греции', 'завер...","['Девушка', 'кокошнике', 'лицо', 'России']",2003,0.285714
3,4,05.11.2003,Известия (Москва),Известия,Не прошло и полвека,Сотрудники МИД Финляндии разыскали живого свид...,"[не, проходить, полвека]","[сотрудник, мид, финляндия, разыскивать, живой...","['Сотрудники', 'МИД', 'Финляндии', 'разыскали'...","['Не', 'прошло', 'полвека']",2003,-0.250000
4,5,06.11.2003,Известия (Москва),Известия,ХРОНИКА,EXXON MOBIL НАМЕРЕНА РАЗВИВАТЬ СОТРУДНИЧЕСТВО ...,[хроника],"[EXXON, MOBIL, намерен, развивать, сотрудничес...","['EXXON', 'MOBIL', 'НАМЕРЕНА', 'РАЗВИВАТЬ', 'С...",['ХРОНИКА'],2003,0.600000
5,6,06.11.2003,Известия (Москва),Известия,"Олег ТВЕРДОВСКИЙ, защитник хоккейного клуба ""А...","Защитник омского ""Авангарда"" Олег ТВЕРДОВСКИЙ ...","[олег, твердовский, защитник, хоккейный, клуб,...","[защитник, омский, ``\n, авангард, ''\n, олег,...","['Защитник', 'омского', '``', 'Авангарда', ""''...","['Олег', 'ТВЕРДОВСКИЙ', 'защитник', 'хоккейног...",2003,0.297297
6,7,06.11.2003,Известия (Москва),Известия,"Семья, здоровье и немного хоккея",Сегодня вечером сборная России сыграет в Хельс...,"[семья, здоровье, немного, хоккей]","[сегодня, вечер, сборная, россия, сыграть, хел...","['Сегодня', 'вечером', 'сборная', 'России', 'с...","['Семья', 'здоровье', 'немного', 'хоккея']",2003,0.545455
7,8,10.11.2003,Известия (Москва),Известия,ХРОНИКА,АВИАЦИОННАЯ СЛУЖБА РОССИИ НЕ СМОГЛА ДОГОВОРИТЬ...,[хроника],"[авиационный, служба, россия, не, смочь, догов...","['АВИАЦИОННАЯ', 'СЛУЖБА', 'РОССИИ', 'НЕ', 'СМО...",['ХРОНИКА'],2003,0.391304
8,9,10.11.2003,Известия (Москва),Известия,"""Хлорка"" уксуса не слаще",Зимой на улицы российских городов выльют более...,"[``\n, хлорка, ''\n, уксус, сладко]","[зима, улица, российский, город, выливать, 1, ...","['Зимой', 'улицы', 'российских', 'городов', 'в...","['``', 'Хлорка', ""''"", 'уксуса', 'слаще']",2003,-0.111111
9,10,10.11.2003,Известия (Москва),Известия,Горящий Евротур,Одержав одну убедительную победу и дважды неуб...,"[гореть, евротур]","[одерживать, один, убедительный, победа, дважд...","['Одержав', 'одну', 'убедительную', 'победу', ...","['Горящий', 'Евротур']",2003,0.030303


In [ ]:
# Plot avg sentiment per year local and regional (if needed)